In [ ]:
from interfaces import *
from utilities import *
import numpy as np
from scipy import stats
from copy import deepcopy
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
#BSpline parameters
n = 19#4
p = 3
# Number of least square points
n_ls = 140

# Open knot vector
knots = np.zeros(n+2*(p+1))

knots = np.linspace(0,1,len(knots))
knots[0:p+1] = 0
knots[-p-1::] = 1
print len(knots), p
vsl = BsplineVectorSpace(p, knots)
print vsl.n_dofs
#BSpline parameters
n += 1 
p = 3
# Number of least square points
n_ls = 140

# Open knot vector
knots = np.zeros(n+2*(p+1))

knots = np.linspace(0,1,len(knots))
knots[0:p+1] = 0
knots[-p-1::] = 1
print len(knots), p
vsl_long = BsplineVectorSpace(p, knots)
print vsl_long.n_dofs

class Antenna(object):
    def __init__(self, cp, vs):
        self.cp = cp
        self.vs = vs
    
    def plot(self, t):
        s = linspace(0,1,50)
        c = self.vs.element(self.cp[:,t,:])
        plot(c(s)[0], c(s)[1])
        plot(self.cp[:,t,0], self.cp[:,t,1], '-or')

    def plot_curve(self, t):
        s = linspace(0,1,50)
        c = self.vs.element(self.cp[:,t,:])
        plot(c(s)[0], c(s)[1])
        #plot(self.cp[:,t,0], self.cp[:,t,1], '-or')
        
    def plot_curve_comparison(self, t, points):
        s = linspace(0,1,50)
        c = self.vs.element(self.cp[:,t,:])
        hold(True)
        plot(c(s)[0], c(s)[1])
        plot(points[:,0], points[:,1], '--')



In [ ]:
# Spiral parameters:
wavenumb = 5.5 #3.5 - 5.5 lunghezza di un passo
length = 11 # 6 - 11
radius = wavenumb / 6
L_1 = 2 * pi * sqrt(radius**2 + (wavenumb/2/pi)**2)
print length, L_1
nturns = length/L_1
heigth = nturns * wavenumb
#nturns = heigth/wavenumb
offset = wavenumb/3
# Spiral analytical expression
cz = lambda x: radius*sin(nturns*2*pi*x)
cy = lambda y: radius*cos(nturns*2*pi*y)
cx = lambda z: heigth*z+offset




In [ ]:
#vs = BsplineVectorSpace(p, knots)

# Least square parameter points
t = linspace(0,1,n_ls) 

# Least square points of the curve
F = array([np.zeros(n_ls+10), np.zeros(n_ls+10), np.zeros(n_ls+10)])
print F.shape, F[:,0]
#F[:,0] = [0.,0.,0.]
print F[:,10:].shape
F[:,10:] = array([cx(t), cy(t), cz(t)])
F[:,1:10] = array([np.linspace(0,F[0,10],9),np.linspace(0,F[1,10],9),np.linspace(0,F[2,10],9)])
print F[:,:10]
# Least square matrix
t = linspace(0,1,n_ls+10)
M = interpolation_matrix(vsl_long, t)

# Control points and curve
CP = lstsq(M, F.T)[0]

print CP.shape
CP[0,:] = [0.,0.,0.]
fig = plt.figure()
ax = fig.gca(projection='3d')
spiral = vsl_long.element(CP)
#theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
x = spiral(t)[0]
y = spiral(t)[1]
z = spiral(t)[2]
#x = F[0,:]
#y = F[1,:]
#z = F[2,:]
print F[0,:10]
print x[:10]
ax.plot(x, y, z, label='parametric curve')
ax.set_xlim3d(0, 10.2)
ax.set_ylim3d(-1,1)
ax.set_zlim3d(-1,1)

ax.legend()



In [ ]:
def spiral_movement(CP_old, CP_new, theta):
#     R = matrix([[1, 0, 0],[0, cos(theta), -sin(theta)],[0, sin(theta), cos(theta)]]
    CP_new[:,0] = CP_old[:,0]
    CP_new[:,1] = cos(theta) * CP_old[:,1] - sin(theta) * CP_old[:,2]
    CP_new[:,2] = cos(theta) * CP_old[:,2] + sin(theta) * CP_old[:,1]


In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
nstep=5
CP_new=zeros([CP.shape[0],nstep,CP.shape[1]])
print CP_new.shape
k=0

for i in np.linspace(0, 2*pi, nstep):
    spiral_movement(CP[:,:], CP_new[:,k,:], i)
    spiral = vsl_long.element(CP_new[:,k,:])
    k+=1
    #theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
    #x = F[0,:]
    x = spiral(t)[0]
    #print x
    #y = F[1,:]
    y = spiral(t)[1]
    #z = F[2,:]
    z = spiral(t)[2]
    ax.plot(x, y, z, label='parametric curve')
ax.set_xlim3d(0, length)
ax.set_ylim3d(-1,1)
ax.set_zlim3d(-1,1)


In [ ]:
arky = ArcLengthParametrizer(vsl_long, CP_new[:,:,:], 2, 1)
CP_new_arclength = arky.reparametrize()


In [ ]:
k=0
fig = plt.figure()
ax = fig.gca(projection='3d')

for i in np.linspace(0, 2*pi, nstep):
    spiral = vsl_long.element(CP_new_arclength[:,k,:])
    k+=1
    #theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
    #x = F[0,:]
    x = spiral(t)[0]
    #print x
    #y = F[1,:]
    y = spiral(t)[1]
    #z = F[2,:]
    z = spiral(t)[2]
    ax.plot(x, y, z, label='parametric curve')
ax.set_xlim3d(0, 1.2)
ax.set_ylim3d(-1,1)
ax.set_zlim3d(-1,1)


In [ ]:
def tangent_continuer(m, point2, new_points, L, Lfin, Np, n_p, direction):
    dl = (Lfin-L)/n_p 
    alpha = np.arctan(m)
    print point2[0]
    if(direction>0):
        dx = dl / np.sqrt(1 + m**2)
    else:
        dx = - dl / np.sqrt(1 + m**2)
    new_points[:,0] = np.linspace(point2[0],point2[0] + n_p * dx, n_p)
    new_points[:,1] = np.linspace(point2[1],point2[1] + n_p * dx * m, n_p)#np.arange(point2[1],point2[1] + m * np * dx, m * dx)
    
    


In [ ]:
# Guasto_CP_rep_try[:,-1,:]=Guasto_CP_rep_try[:,0,:]
# np.save('/Users/nicolagiuliani/giuliani-code/blender/CP_Guasto_rep', Guasto_CP_rep_try[:,:,:])
# ant_try.plot(0)
# ant_try.plot(12)
# #ant_try.plot(12)